Join the data from Part 1 with the data from Part 2 to create a new dataframe.

In [ ]:
# Import necessary libraries
import pandas as pd  # For handling and analyzing data
import matplotlib.pyplot as plt  # For creating visualizations
import random  # For selecting random stations

# Step 1: Load the datasets
# Load the bike station data (created in Part 1)
stations_df = pd.read_csv("luxembourg_bike_stations.csv")
print("Bike Stations DataFrame:")
print(stations_df.head())  # Display the first few rows for verification

# Load the Foursquare POI data (created in Part 2)
foursquare_df = pd.read_csv(r"C:\Users\Admin\Downloads\statistical modelling project\-Statistical-Modelling-Project\notebooks\foursquare_pois.csv")
print("\nFoursquare POIs DataFrame:")
print(foursquare_df.head())  # Display the first few rows for verification

# Load the Yelp POI data (created in Part 2)
yelp_df = pd.read_csv(r"C:\Users\Admin\Downloads\statistical modelling project\-Statistical-Modelling-Project\notebooks\yelp_pois.csv")
print("\nYelp POIs DataFrame:")
print(yelp_df.head())  # Display the first few rows for verification

# Step 2: Merge datasets
# Merge Foursquare data with bike station data on the 'station_name' column
merged_foursquare = pd.merge(
    stations_df,  # Bike station data
    foursquare_df,  # Foursquare POI data
    on="station_name",  # Common column for merging
    how="left"  # Retain all bike station data, even if no matching POI data exists
)

# Merge Yelp data with bike station data on the 'station_name' column
merged_yelp = pd.merge(
    stations_df,  # Bike station data
    yelp_df,  # Yelp POI data
    on="station_name",  # Common column for merging
    how="left"  # Retain all bike station data, even if no matching POI data exists
)

# Display the first few rows of the merged data for Foursquare
print("\nMerged Data (Foursquare):")
print(merged_foursquare.head())

# Display the first few rows of the merged data for Yelp
print("\nMerged Data (Yelp):")
print(merged_yelp.head())

Provide a visualization that you used as part of your EDA process. Explain the initial pattern or relationship you discoved through this visualization. 

In [ ]:
##Step 3: Visualization - Randomly Select 10 Stations
# Get a random sample of 10 stations
random_stations = random.sample(list(merged_yelp["station_name"].unique()), 10)

# Filter the dataset for these random stations
filtered_yelp = merged_yelp[merged_yelp["station_name"].isin(random_stations)]

# Count the number of POIs for these stations
poi_count_random = filtered_yelp.groupby("station_name")["poi_name"].count()

# Create a bar plot for the number of POIs near the randomly selected stations
plt.figure(figsize=(10, 6))
poi_count_random.plot(kind="bar", color="orange")
plt.title("Number of POIs for Randomly Selected Stations (Yelp)")
plt.xlabel("Station Name")
plt.ylabel("Number of POIs")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()  # Adjust layout to prevent overlap
plt.show()

# Step 4: Visualization - POI Count vs. Available Bikes
# Scatter plot showing the relationship between POI count and available bikes
plt.figure(figsize=(8, 5))
plt.scatter(
    merged_yelp["available_bikes"],  # Available bikes
    merged_yelp.groupby("station_name")["poi_name"].count(),  # Number of POIs
    alpha=0.7,  # Transparency of points
    color="blue"  # Color of points
)
plt.title("Relationship Between POI Count and Available Bikes")
plt.xlabel("Number of Available Bikes")
plt.ylabel("Number of POIs (Yelp)")
plt.grid(True)
plt.show()

# Step 5: Visualization - Average POI Ratings per Station
# Calculate the average POI rating near each station
average_ratings = merged_yelp.groupby("station_name")["rating"].mean()

# Create a bar plot for the average POI ratings
plt.figure(figsize=(10, 6))
average_ratings.plot(kind="bar", color="green")
plt.title("Average POI Ratings Near Each Station (Yelp)")
plt.xlabel("Station Name")
plt.ylabel("Average Rating")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

In [ ]:
# Import necessary libraries
import sqlite3  # For SQLite database creation and management
import os  # For directory management
import pandas as pd  # For handling data

# Step 1: Ensure the data/ directory exists
data_dir = "data"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)  # Create the directory if it doesn't exist

# Step 2: Define the SQLite database path
db_path = os.path.join(data_dir, "bike_sharing_pois.db")

# Step 3: Establish a connection to the SQLite database
conn = sqlite3.connect(db_path)  # Creates or connects to the SQLite database
cursor = conn.cursor()  # Create a cursor object to execute SQL commands

# Step 4: Load DataFrames
# Replace these paths with the actual paths to your CSV files
stations_df = pd.read_csv("luxembourg_bike_stations.csv")  # Bike station data
foursquare_df = pd.read_csv(r"C:\Users\Admin\Downloads\statistical modelling project\-Statistical-Modelling-Project\notebooks\foursquare_pois.csv") # Foursquare POI data
yelp_df = pd.read_csv(r"C:\Users\Admin\Downloads\statistical modelling project\-Statistical-Modelling-Project\notebooks\yelp_pois.csv")  # Yelp POI data

# Step 5: Save DataFrames to the SQLite database
stations_df.to_sql("stations_data", conn, if_exists="replace", index=False)  # Save bike station data
foursquare_df.to_sql("foursquare_data", conn, if_exists="replace", index=False)  # Save Foursquare POI data
yelp_df.to_sql("yelp_data", conn, if_exists="replace", index=False)  # Save Yelp POI data

# Step 6: Validate data stored in the database
# Fetch some rows from the Stations table
print("\nSample rows from the Stations table:")
print(pd.read_sql("SELECT * FROM stations_data LIMIT 5;", conn))

# Fetch some rows from the Foursquare table
print("\nSample rows from the Foursquare table:")
print(pd.read_sql("SELECT * FROM foursquare_data LIMIT 5;", conn))

# Fetch some rows from the Yelp table
print("\nSample rows from the Yelp table:")
print(pd.read_sql("SELECT * FROM yelp_data LIMIT 5;", conn))

# Step 7: Close the database connection
conn.close()  # Close the connection to ensure no further changes

# Print confirmation
print(f"\nDatabase successfully created and saved at: {db_path}")



Sample rows from the Stations table:
                     station_name   latitude  longitude  available_bikes
0  #00126-SALLES DES FÊTES HOLZEM  49.615901   5.993194                9
1                    #00109-FOYER  49.569178   6.081750                2
2     #00053-LUDWIG VAN BEETHOVEN  49.590000   6.117200                2
3                #00013-GELLE FRA  49.609510   6.128503                4
4         #00116-CHARLY'S STATION  49.649823   6.225796                2

Sample rows from the Foursquare table:
                     station_name                poi_name  \
0  #00126-SALLES DES FÊTES HOLZEM       Restaurant Papaya   
1                    #00109-FOYER  Burger King Leudelange   
2                    #00109-FOYER      Il Punto Delizioso   
3                    #00109-FOYER                 El Gato   
4                    #00109-FOYER    Les Espaces Réunions   

               category   latitude  longitude  
0      Sushi Restaurant  49.616679   5.993284  
1  Fast Food Restaura

Look at the data before and after the join to validate your data.